Daily Challenge: Simplified Self-Attention Explained

1. Simplified self-attention

In [1]:
import torch

In [2]:
inputs = torch.tensor(
[
    [0.43, 0.15, 0.89], # your
    [0.55, 0.87, 0.66], # journey
    [0.57, 0.85, 0.64], # starts
    [0.22, 0.58, 0.33], # with
    [0.77, 0.25, 0.10], # one
    [0.05, 0.80, 0.55] # step
]
)

In [3]:
# connaitre la shape du tensor
inputs.shape

torch.Size([6, 3])

1.1 Computing Attention Weights for Inputs[2]:

query vector

In [4]:
# Sélection du Query (3e mot = "starts")
query = inputs[2]

# Afficher le Query sélectionné (ce=vecteur du mot)
print(query)

tensor([0.5700, 0.8500, 0.6400])


1.1.1 Attention Score:

attention score : mesure la similarité entre le Query (mot sélectionné) et les autres mots.plus le score est élevé, plus les mots sont liés

In [5]:
attn_scores_2 = torch.empty(inputs.shape[0])
attn_scores_2

tensor([2.1707e-18, 7.0952e+22, 1.7748e+28, 1.8176e+31, 7.2708e+31, 5.0778e+31])

In [6]:
# Initialiser un tableau pour stocker les scores d'attention
attn_scores_2 = torch.zeros(inputs.shape[0])
attn_scores_2

tensor([0., 0., 0., 0., 0., 0.])

In [7]:
# Calcul des scores d’attention (produit scalaire entre Query et chaque mot)
for i, x_i in enumerate(inputs):
  # On compare chaque mot (x_i) avec "starts" (query). x_i est donc le vecteur d’un mot de la phrase
    attn_scores_2[i] = torch.dot(x_i, query)


 Ce code calcule le produit scalaire entre "starts" et chaque mot de la phrase.

In [8]:
# Afficher les scores d'attention
print(f"\nScores d'attention pour le mot 'starts' :\n{attn_scores_2}")


Scores d'attention pour le mot 'starts' :
tensor([0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605])


-->Plus le score est élevé, plus le mot est proche de "starts"

1.1.2 Attention Weights:

In [9]:
import torch.nn.functional as F  # Pour utiliser Softmax


Softmax fait exactement cela :Il prend une liste de nombres et les transforme en probabilités. La somme des probabilités est toujours égale à 1.

In [10]:
attn_weights_2 = torch.softmax(attn_scores_2, dim=0)
attn_weights_2

tensor([0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565])

In [11]:
# Appliquer la fonction Softmax sur les scores d'attention
attn_weights_2 = F.softmax(attn_scores_2, dim=0)
#  Après Softmax, ils deviendront des probabilités

In [12]:
# Afficher les poids d'attention (après Softmax)
print(f"\n Poids d'attention après Softmax :\n{attn_weights_2}")
# Vérifie que la somme des poids est bien égale à 1 (normalisation):
print(f"\n Somme des poids d'attention : {attn_weights_2.sum()} (doit être ≈ 1)")


 Poids d'attention après Softmax :
tensor([0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565])

 Somme des poids d'attention : 1.0000001192092896 (doit être ≈ 1)


- 1.1.3 Context Vector:

contexte vector est une combinaison pondérée des mots selon leurs poids d’attention.
➡ Il permet de résumer toutes les informations importantes d’une phrase en un seul vecteur

Ensuite, on fait la somme des résultats pour obtenir le contexte final.
Formule :
           context_vector= ∑(attn_weights*inputs)

In [13]:
# Calcul du vecteur de contexte comme somme pondérée des mots
context_vector = torch.sum(attn_weights_2.unsqueeze(1) * inputs, dim=0)

In [14]:
# Afficher le vecteur de contexte
print(context_vector)

tensor([0.4431, 0.6496, 0.5671])


Résumé : Converti les mots en vecteurs (embeddings)./Choisi un mot Query pour calculer l’attention./Calculé les scores d’attention avec le produit scalaire./Appliqué Softmax pour obtenir des poids d’attention normalisés./Calculé le vecteur de contexte, qui représente toute la phrase.

Self-Attention est essentiel dans les Transformers : Self-Attention analyse tous les mots en même temps; \Les Transformers utilisent plusieurs couches de Self-Attention en même temps (on appelle cela Multi-Head Attention)

1.2 Computing Attention Weights for All Inputs:

Implémentation : Calculer les Scores d’Attention pour Tous les Mots

In [15]:
# Calcul de la matrice des scores d'attention
attn_scores_matrix = torch.matmul(inputs, inputs.T)  # Produit matriciel entre les mots

In [16]:
# Affichage de la matrice des scores
print(attn_scores_matrix)
# Cela va donner une "matrice d’attention", une table qui montre les relations entre chaque mot.
# Chaque cellule de cette matrice représente le score d’attention entre deux mots.

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


On applique Softmax sur chaque ligne de la matrice (dim=1) pour obtenir les poids d’attention.

In [17]:
import torch.nn.functional as F  # Pour utiliser Softmax

In [18]:
# Appliquer Softmax pour normaliser les scores d’attention
attn_weights_matrix = F.softmax(attn_scores_matrix, dim=1)

In [19]:
# Affichage de la matrice des poids d'attention
print(f"\nMatrice des poids d'attention (après Softmax) :\n{attn_weights_matrix}")
print(f"\n Somme des poids d'attention par ligne :\n{attn_weights_matrix.sum(dim=1)} (doit être ≈ 1)")


Matrice des poids d'attention (après Softmax) :
tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])

 Somme des poids d'attention par ligne :
tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000]) (doit être ≈ 1)


In [20]:
# Calcul des context vectors pour chaque mot / multiplier la matrice des poids d’attention par nos embeddings d’entrée.
context_vectors = torch.matmul(attn_weights_matrix, inputs)

In [21]:
# Affichage des context vectors
print(context_vectors)

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])


Résumé du processus complet du Self-Attention: 1.Converti les mots en vecteurs (embeddings). 2.Calculé les scores d’attention entre tous les mots (produit scalaire).3.Appliqué Softmax pour normaliser les scores en probabilités.4.Généré les context vectors en combinant chaque mot avec ses voisins.

2. The ‘Self’ in Self-Attention¶

2.2.1 Initialiser les trois matrices de poids Wq, Wk, Wv :

Implémentation : Initialisation des Matrices Wq, Wk, Wv

dans les Transformers, ces embeddings sont d’abord transformés par des matrices de poids Wq, Wk, Wv.
Wq (Weights for Queries) → Transforme un mot en une question pour chercher l’information pertinente.
Wk (Weights for Keys) → Transforme un mot en référence à laquelle on compare les questions.
Wv (Weights for Values) → Transforme un mot en information finale qui sera transmise.
initialise ces matrices comme des poids entraînables en PyTorch

In [22]:
# Définir la dimension des embeddings
embedding_dim = inputs.shape[1]  # Ici, nos embeddings ont 3 dimensions

In [23]:
# Initialiser les matrices Wq, Wk, Wv (3x3 car on a 3 dimensions d'embeddings)
Wq = torch.nn.Parameter(torch.rand(embedding_dim, embedding_dim))
Wk = torch.nn.Parameter(torch.rand(embedding_dim, embedding_dim))
Wv = torch.nn.Parameter(torch.rand(embedding_dim, embedding_dim))
# Ce code crée les trois matrices Wq, Wk, Wv qui serviront à transformer nos mots
# Ces matrices vont apprendre, au fil de l'entraînement, comment transformer correctement les mots en Q, K et V.

In [24]:
# Afficher les matrices
print(f"Matrice Wq :\n{Wq}")
print(f"Matrice Wk :\n{Wk}")
print(f"Matrice Wv :\n{Wv}")
# Ces matrices seront utilisées pour transformer nos embeddings de mots en Query, Key et Value

Matrice Wq :
Parameter containing:
tensor([[0.8713, 0.5234, 0.9295],
        [0.5059, 0.8670, 0.0284],
        [0.1292, 0.1072, 0.9899]], requires_grad=True)
Matrice Wk :
Parameter containing:
tensor([[0.8595, 0.2962, 0.6314],
        [0.3953, 0.2804, 0.5433],
        [0.0554, 0.0690, 0.4600]], requires_grad=True)
Matrice Wv :
Parameter containing:
tensor([[0.0840, 0.9858, 0.9330],
        [0.5648, 0.4157, 0.6031],
        [0.5617, 0.1062, 0.0103]], requires_grad=True)


dans les Transformers, ces embeddings sont d’abord transformés par des matrices de poids Wq, Wk, Wv.\
Wq (Weights for Queries) → Transforme un mot en une question pour chercher l’information pertinente.\
Wk (Weights for Keys) → Transforme un mot en référence à laquelle on compare les questions.\
Wv (Weights for Values) → Transforme un mot en information finale qui sera transmise.\
 initialise ces matrices comme des poids entraînables en PyTorch

-->Ces matrices seront utilisées pour transformer nos embeddings de mots en Query, Key et Value !

In [25]:
# Implémentation : Calcul des Q, K, V pour tous les mots
# Calcul des Query, Key et Value en multipliant les inputs par Wq, Wk, Wv
queries = torch.matmul(inputs, Wq)
keys = torch.matmul(inputs, Wk)
values = torch.matmul(inputs, Wv)

# Affichage des résultats
print(f"\n Queries (Q) :\n{queries}")
print(f"\n Keys (K) :\n{keys}")
print(f"\n Values (V) :\n{values}")


 Queries (Q) :
tensor([[0.5656, 0.4505, 1.2850],
        [1.0046, 1.1129, 1.1893],
        [1.0094, 1.1039, 1.1875],
        [0.5278, 0.6534, 0.5476],
        [0.8103, 0.6305, 0.8218],
        [0.5194, 0.7787, 0.6137]], grad_fn=<MmBackward0>)

 Keys (K) :
tensor([[0.4782, 0.2308, 0.7624],
        [0.8532, 0.4524, 1.1236],
        [0.8614, 0.4513, 1.1161],
        [0.4366, 0.2506, 0.6058],
        [0.7661, 0.3051, 0.6680],
        [0.3897, 0.2771, 0.7192]], grad_fn=<MmBackward0>)

 Values (V) :
tensor([[0.6208, 0.5808, 0.5008],
        [0.9083, 0.9740, 1.0446],
        [0.8874, 0.9832, 1.0510],
        [0.5314, 0.4930, 0.5584],
        [0.2620, 0.8736, 0.8702],
        [0.7650, 0.4403, 0.5348]], grad_fn=<MmBackward0>)


2.2.2 Calculer les vecteurs de requête, de clé et de valeur pour les entrées[1] :

Transformer inputs[1] en Q, K et V

In [26]:
# Sélectionner le mot `inputs[1]` (2e mot de la phrase)
word_1 = inputs[1]  # Ce mot représente "journey" dans notre dataset

# Transformer en Query, Key et Value
query_1 = torch.matmul(word_1, Wq)
key_1 = torch.matmul(word_1, Wk)
value_1 = torch.matmul(word_1, Wv)

In [27]:
print(f"\n Query (Q) pour inputs[1] :\n{query_1}")
print(f"\n Key (K) pour inputs[1] :\n{key_1}")
print(f"\nValue (V) pour inputs[1] :\n{value_1}")


🔍 Query (Q) pour inputs[1] :
tensor([1.0046, 1.1129, 1.1893], grad_fn=<SqueezeBackward4>)

🔑 Key (K) pour inputs[1] :
tensor([0.8532, 0.4524, 1.1236], grad_fn=<SqueezeBackward4>)

📦 Value (V) pour inputs[1] :
tensor([0.9083, 0.9740, 1.0446], grad_fn=<SqueezeBackward4>)


2.2.3 Calculez les entrées du score d'attention[1][1] ou ω11 :

mesurer la similarité entre query_1 et key_1\ns utiliserons le produit scalaire (dot product) pour mesurer cette similarité.

In [28]:
# Calcul du score d’attention entre inputs[1] et lui-même (ω₁₁)
attn_score_11 = torch.dot(query_1, key_1)

In [29]:
# Afficher le résultat
print(attn_score_11)

tensor(2.6969, grad_fn=<DotBackward0>)


2.2.4 Calculer tous les scores d'attention pour les entrées[1] :

In [30]:
# Calcul des scores d'attention entre inputs[1] et TOUS les autres mots
attn_scores_1 = torch.matmul(query_1, keys.T)

In [31]:
print (attn_scores_1)
# Chaque valeur indique la force de la relation entre inputs[1] et un autre mot.

tensor([1.6440, 2.6969, 2.6950, 1.4380, 1.9037, 1.5552],
       grad_fn=<SqueezeBackward4>)


2.2.5 Pondérations d'attention pour les entrées[1] :

In [32]:
import torch.nn.functional as F  # Pour utiliser Softmax

In [33]:
# Appliquer Softmax pour normaliser les scores d'attention
attn_weights_1 = F.softmax(attn_scores_1, dim=0)

In [35]:
# Afficher les poids d'attention normalisés
print (attn_weights_1)

tensor([0.1025, 0.2939, 0.2933, 0.0835, 0.1330, 0.0938],
       grad_fn=<SoftmaxBackward0>)


In [37]:
print(attn_weights_1.sum())

tensor(1., grad_fn=<SumBackward0>)


2.2.6 Calculer le vecteur de contexte pour les entrées[1] :

vecteur de contexte est un résumé de toute l’information importante pour un mot donné (inputs[1]).

In [39]:
# Calcul du vecteur de contexte comme somme pondérée des valeurs (V)
context_vector_1 = torch.sum(attn_weights_1.unsqueeze(1) * values, dim=0)

# Afficher le vecteur de contexte
print(f"\n Vecteur de Contexte pour inputs[1] :\n{context_vector_1}")
# Ce vecteur unique de 3 dimensions représente maintenant toute la phrase du point de vue du mot inputs[1] !


 Vecteur de Contexte pour inputs[1] :
tensor([0.7419, 0.8328, 0.8791], grad_fn=<SumBackward1>)


2.3 Calcul des paramètres de pondération pour toutes les entrées :

In [42]:
# 1️⃣ Définition des dimensions et Initialisation des Matrices Wq, Wk, Wv
embedding_dim = inputs.shape[1]  # 3 dimensions pour chaque mot
Wq = torch.nn.Parameter(torch.rand(embedding_dim, embedding_dim))  # Matrice pour Query
Wk = torch.nn.Parameter(torch.rand(embedding_dim, embedding_dim))  # Matrice pour Key
Wv = torch.nn.Parameter(torch.rand(embedding_dim, embedding_dim))  # Matrice pour Value

# 2️⃣ Transformation des mots en Query (Q), Key (K) et Value (V)
queries = torch.matmul(inputs, Wq)  # Appliquer Wq sur les mots
keys = torch.matmul(inputs, Wk)     # Appliquer Wk sur les mots
values = torch.matmul(inputs, Wv)   # Appliquer Wv sur les mots

# 3️⃣ Calcul de la Matrice des Scores d’Attention (Produit scalaire Q * K^T)
attn_scores = torch.matmul(queries, keys.T)

# 4️⃣ Normalisation des Scores avec Softmax pour obtenir les Poids d’Attention
attn_weights = F.softmax(attn_scores, dim=1)

# 5️⃣ Calcul des Vecteurs de Contexte (somme pondérée des valeurs)
context_vectors = torch.matmul(attn_weights, values)

# 6️⃣ Affichage des résultats
print(f"\n🔍 Matrice des Queries (Q) :\n{queries}")
print(f"\n🔑 Matrice des Keys (K) :\n{keys}")
print(f"\n📦 Matrice des Values (V) :\n{values}")
print(f"\n📊 Matrice des Scores d’Attention (avant Softmax) :\n{attn_scores}")
print(f"\n🎯 Matrice des Poids d’Attention (après Softmax) :\n{attn_weights}")
print(f"\n📌 Matrice des Vecteurs de Contexte :\n{context_vectors}")


🔍 Matrice des Queries (Q) :
tensor([[1.3146, 0.8511, 0.8483],
        [1.6573, 0.8360, 0.9098],
        [1.6471, 0.8177, 0.9169],
        [0.8609, 0.4420, 0.4164],
        [1.0012, 0.2578, 0.7865],
        [1.0219, 0.6512, 0.3851]], grad_fn=<MmBackward0>)

🔑 Matrice des Keys (K) :
tensor([[0.4441, 0.7701, 0.7344],
        [0.9655, 0.9465, 0.6822],
        [0.9468, 0.9366, 0.6785],
        [0.5895, 0.5001, 0.3222],
        [0.3427, 0.4962, 0.4212],
        [0.8030, 0.6308, 0.3857]], grad_fn=<MmBackward0>)

📦 Matrice des Values (V) :
tensor([[0.7897, 0.5539, 0.3574],
        [0.9177, 1.0407, 0.4663],
        [0.9033, 1.0475, 0.4683],
        [0.4922, 0.5463, 0.2303],
        [0.3886, 0.8753, 0.3729],
        [0.6665, 0.5190, 0.2289]], grad_fn=<MmBackward0>)

📊 Matrice des Scores d’Attention (avant Softmax) :
tensor([[1.8623, 2.6535, 2.6174, 1.4739, 1.2301, 1.9196],
        [2.0481, 3.0120, 2.9694, 1.6882, 1.3660, 2.2090],
        [2.0347, 2.9897, 2.9474, 1.6753, 1.3564, 2.1920],
       

In [41]:
# 5️⃣ Calcul des Vecteurs de Contexte (somme pondérée des valeurs)
context_vectors = torch.matmul(attn_weights, values)

# 6️⃣ Affichage des résultats
print(f"\n🔍 Matrice des Queries (Q) :\n{queries}")
print(f"\n🔑 Matrice des Keys (K) :\n{keys}")
print(f"\n📦 Matrice des Values (V) :\n{values}")
print(f"\n📊 Matrice des Scores d’Attention (avant Softmax) :\n{attn_scores}")
print(f"\n🎯 Matrice des Poids d’Attention (après Softmax) :\n{attn_weights}")
print(f"\n📌 Matrice des Vecteurs de Contexte :\n{context_vectors}")


🔍 Matrice des Queries (Q) :
tensor([[0.5656, 0.4505, 1.2850],
        [1.0046, 1.1129, 1.1893],
        [1.0094, 1.1039, 1.1875],
        [0.5278, 0.6534, 0.5476],
        [0.8103, 0.6305, 0.8218],
        [0.5194, 0.7787, 0.6137]], grad_fn=<MmBackward0>)

🔑 Matrice des Keys (K) :
tensor([[0.4782, 0.2308, 0.7624],
        [0.8532, 0.4524, 1.1236],
        [0.8614, 0.4513, 1.1161],
        [0.4366, 0.2506, 0.6058],
        [0.7661, 0.3051, 0.6680],
        [0.3897, 0.2771, 0.7192]], grad_fn=<MmBackward0>)

📦 Matrice des Values (V) :
tensor([[0.6208, 0.5808, 0.5008],
        [0.9083, 0.9740, 1.0446],
        [0.8874, 0.9832, 1.0510],
        [0.5314, 0.4930, 0.5584],
        [0.2620, 0.8736, 0.8702],
        [0.7650, 0.4403, 0.5348]], grad_fn=<MmBackward0>)

📊 Matrice des Scores d’Attention (avant Softmax) :
tensor([[1.3541, 2.1301, 2.1247, 1.1383, 1.4291, 1.2694],
        [1.6440, 2.6969, 2.6950, 1.4380, 1.9037, 1.5552],
        [1.6429, 2.6948, 2.6931, 1.4368, 1.9034, 1.5533],
       